# Determining best combination of preprocessing techniques  

## Loading Libraries

In [1]:

#MÅ GÅ NØYE IGJENNOM Å SJEKKE HVA SOM FAKTISK BRUKES

%load_ext autoreload
%autoreload 2

import random as rn
import tensorflow as tf
from keras import backend as K

# external imports
import numpy as np
import pandas as pd
import seaborn as sb
import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
import pylab as pl
import matplotlib.pyplot as plt
import csv
import scipy
import os.path
import pickle

import keras
from keras.layers import *
from keras.layers.core import *
from keras import backend as K

import sklearn as sk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import ParameterGrid
import time

# internal imports
import helpers as HL
import cleaning as CL
import glove_module as GV
import neural_nets as NN
import tokenizing as TO

import maketextfile as MT



/Users/havardbjornoy/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


## Defining Data Paths 

### Files used to create model:

In [2]:
# Constants
#BRUKER VI DET HER??
DATA_FOLDER = os.path.join("glove.twitter.27B") 
DATA_25DIM = DATA_FOLDER + "/glove.twitter.27B.25d.txt"
DATA_50DIM = DATA_FOLDER + "/glove.twitter.27B.50d.txt"
DATA_100DIM = DATA_FOLDER + "/glove.twitter.27B.100d.txt"
DATA_200DIM = DATA_FOLDER + "/glove.twitter.27B.200d.txt"

### Data files:

In [3]:
training_set_pos = "train_pos.txt" 
training_set_neg = "train_neg.txt"
training_set_pos_full = "train_pos_full.txt"
training_set_neg_full = "train_neg_full.txt"
test_set = "test_data.txt"

## Import pretrained GloVe with gensim
one can use gensims word2vec functions to check similarity and other interesting functions https://radimrehurek.com/gensim/models/word2vec.html

## Create the word embeddings using the created gensim-.txt file.

In [4]:
#Pick one, the higher dimension, the better result and longer computational time. 

#global_vectors=GV.make_glove("data/gensim_global_vectors_25dim.txt")
#global_vectors=GV.make_glove("gensim_global_vectors_50dim.txt")
#global_vectors=GV.make_glove("gensim_global_vectors_100dim.txt")
global_vectors=GV.make_glove("gensim_data_folder/gensim_glove_vectors_200dim.txt")

## Creating corpus:

In [5]:
#When testing
inputfiles=[training_set_pos,training_set_neg,test_set]

#when using full data set:
#inputfiles=[training_set_pos_full,training_set_neg_full,test_set]

full_corpus, file_lengths=HL.create_corpus(inputfiles)
nr_pos_tweets = file_lengths[0]
nr_neg_tweets = file_lengths[1]
total_training_tweets =file_lengths [0]+file_lengths[1]

## Picking the neural net

In [6]:
neural_nets=[NN.deep_HB]

## Initializing variables to apply all preprocessing techniques:

In [7]:
#Initializing vectors:

corpuses=[]
corpuses.append(full_corpus)


In [8]:
#Defining names of corpuses: 
names=['original_corpus','SH_corpus','SHM_corpus','H_corpus','HK_corpus','PS_corpus','NS__corpus','OS_corpus','N_corpus','NM_corpus','ST_corpus','SP_corpus','E_corpus','SN_corpus','RS_corpus','N-2_corpus','N-3_corpus','N-4_corpus']

In [9]:
#Defining inputs to preprocessing function: 
inputs=[{'hashtag': True, 'segmentation_hash': True},
        {'hashtag':True,'segmentation_hash': True,'hashtag_mention':True},
        {'hearts':True},
        {'hugs_and_kisses':True},
        {'pos_smilies':True},
        {'neg_smilies':True},
        {'other_smilies':True},
        {'numbers':True},
        {'numbers':True,'number_mention':True},
        {'stemming':True},
        {'spelling':True},#Warning: When True, it takes forever. Recomended to always have as false 
        {'elongation':True},
        {'set_to_not':True},
        {'remove_signs':True}]

## Applying all preprocessing techniques to the original corpus: 

In [10]:

for input_ in inputs: 
        corpus=TO.preprocess_corpus(full_corpus, **input_)
        corpuses.append(corpus)
        

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Time in min before starting first for loop: 0.1617306669553121
Time in min after 1  tweets: 0.16198389530181884
Time in min after 25001  tweets: 0.30229872862497964
Time in min after 50001  tweets: 0.41871701876322426
Time in min after 75001  tweets: 0.5374351302782695
Time in min after 100001  tweets: 0.6650677680969238
Time in min after 125001  tweets: 0.7544985810915629
Time in min after 150001  tweets: 0.8679561018943787
Time in min after 175001  tweets: 0.9501758495966593
Time in min after 200001  tweets: 1.0365543007850646
Time in min total: 1.0756685972213744
Reading twitter - 1grams ...
Reading twitter - 2grams ...
Time in min before starting first for loop: 0.16107724905014037
Time in min after 1  tweets: 0.1611544688542684
Time in min after 25001  tweets: 0.3606411655743917
Time in min after 50001  tweets: 0.4720538338025411
Time in min after 75001  tweets: 0.5838240186373392
Time in min after 100001  tweets: 0.6974303

Time in min after 150001  tweets: 0.20171878337860108
Time in min after 175001  tweets: 0.2400106986363729
Time in min after 200001  tweets: 0.2774257024129232
Time in min total: 0.2929166316986084


In [11]:
ns=[2,3,4]
for n in ns: 
    corpus=HL.creating_n_grams_corpus(n,full_corpus)
    corpuses.append(corpus)

## Round 1: Testing all preprocessing techniques: 

In [12]:
accuracies=[]
stds=[]

for corpus in corpuses: 
    model_score=GV.classify_with_neural_networks(neural_nets, global_vectors, corpus, total_training_tweets, nr_pos_tweets, epochs=100, n_folds=3)
    accuracies.append(model_score[0][0])
    stds.append(model_score[0][1])

tweets processed: 0  of total number of tweets: 200000
tweets processed: 50000  of total number of tweets: 200000
tweets processed: 100000  of total number of tweets: 200000
tweets processed: 150000  of total number of tweets: 200000
Train on 133332 samples, validate on 66668 samples
Epoch 1/100
133332/133332 [==============================] - 7s 51us/step - loss: 0.4925 - acc: 0.7579 - val_loss: 0.4089 - val_acc: 0.8067
Epoch 2/100
133332/133332 [==============================] - 5s 35us/step - loss: 0.3977 - acc: 0.8128 - val_loss: 0.3935 - val_acc: 0.8165
Epoch 3/100
133332/133332 [==============================] - 4s 33us/step - loss: 0.3836 - acc: 0.8212 - val_loss: 0.3892 - val_acc: 0.8183
Epoch 4/100
133332/133332 [==============================] - 4s 32us/step - loss: 0.3729 - acc: 0.8268 - val_loss: 0.3860 - val_acc: 0.8200
Epoch 5/100
133332/133332 [==============================] - 4s 31us/step - loss: 0.3632 - acc: 0.8325 - val_loss: 0.3806 - val_acc: 0.8232
Epoch 6/100
133

133334/133334 [==============================] - 4s 32us/step - loss: 0.3472 - acc: 0.8403 - val_loss: 0.3776 - val_acc: 0.8235
Epoch 7/100
133334/133334 [==============================] - 4s 31us/step - loss: 0.3413 - acc: 0.8449 - val_loss: 0.3771 - val_acc: 0.8259
Epoch 8/100
133334/133334 [==============================] - 4s 31us/step - loss: 0.3313 - acc: 0.8493 - val_loss: 0.3805 - val_acc: 0.8244
Epoch 00008: early stopping
66666/66666 [==============================] - 3s 43us/step
Model:  deep_HB
[0.82583848322675923, 0.82397323973060921, 0.82436324363601265]
0.82% (+/- 0.00%)
Negative sentiment: 79.10%  Positive sentiment: 85.84%
Percentage of positive classifications (should be 50%ish): 53.3715144405
Time taken:  2.5824588179588317 

tweets processed: 0  of total number of tweets: 200000
tweets processed: 50000  of total number of tweets: 200000
tweets processed: 100000  of total number of tweets: 200000
tweets processed: 150000  of total number of tweets: 200000
Train on 1

133334/133334 [==============================] - 6s 46us/step - loss: 0.4920 - acc: 0.7637 - val_loss: 0.4107 - val_acc: 0.8053
Epoch 2/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3946 - acc: 0.8154 - val_loss: 0.3967 - val_acc: 0.8141
Epoch 3/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3803 - acc: 0.8228 - val_loss: 0.3908 - val_acc: 0.8159
Epoch 4/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3698 - acc: 0.8286 - val_loss: 0.3857 - val_acc: 0.8180
Epoch 5/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3605 - acc: 0.8338 - val_loss: 0.3828 - val_acc: 0.8197
Epoch 6/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3525 - acc: 0.8385 - val_loss: 0.3823 - val_acc: 0.8213
Epoch 7/100
133334/133334 [==============================] - 5s 34us/step - loss: 0.3463 - acc: 0.8419 - val_loss: 0.3834 - val_acc: 0.8206
Epoch 8/100
133334/133334 [=====

Epoch 5/100
133334/133334 [==============================] - 4s 34us/step - loss: 0.3623 - acc: 0.8334 - val_loss: 0.3816 - val_acc: 0.8207
Epoch 6/100
133334/133334 [==============================] - 4s 32us/step - loss: 0.3532 - acc: 0.8378 - val_loss: 0.3841 - val_acc: 0.8201
Epoch 7/100
133334/133334 [==============================] - 4s 32us/step - loss: 0.3465 - acc: 0.8408 - val_loss: 0.3851 - val_acc: 0.8211
Epoch 8/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3383 - acc: 0.8459 - val_loss: 0.3857 - val_acc: 0.8213
Epoch 00008: early stopping
66666/66666 [==============================] - 4s 59us/step
Train on 133334 samples, validate on 66666 samples
Epoch 1/100
133334/133334 [==============================] - 6s 48us/step - loss: 0.4916 - acc: 0.7642 - val_loss: 0.4102 - val_acc: 0.8057
Epoch 2/100
133334/133334 [==============================] - 4s 32us/step - loss: 0.3945 - acc: 0.8150 - val_loss: 0.3963 - val_acc: 0.8141
Epoch 3/100
133334/13

Epoch 7/100
133332/133332 [==============================] - 4s 32us/step - loss: 0.3485 - acc: 0.8415 - val_loss: 0.3815 - val_acc: 0.8232
Epoch 8/100
133332/133332 [==============================] - 4s 33us/step - loss: 0.3411 - acc: 0.8446 - val_loss: 0.3826 - val_acc: 0.8229
Epoch 00008: early stopping
66668/66668 [==============================] - 5s 70us/step
Train on 133334 samples, validate on 66666 samples
Epoch 1/100
133334/133334 [==============================] - 6s 48us/step - loss: 0.4914 - acc: 0.7637 - val_loss: 0.4067 - val_acc: 0.8090
Epoch 2/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3965 - acc: 0.8139 - val_loss: 0.3928 - val_acc: 0.8158
Epoch 3/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3830 - acc: 0.8212 - val_loss: 0.3898 - val_acc: 0.8167
Epoch 4/100
133334/133334 [==============================] - 5s 34us/step - loss: 0.3706 - acc: 0.8276 - val_loss: 0.3879 - val_acc: 0.8182
Epoch 5/100
133334/13

133332/133332 [==============================] - 17s 129us/step - loss: 0.4925 - acc: 0.7579 - val_loss: 0.4089 - val_acc: 0.8067
Epoch 2/100
133332/133332 [==============================] - 4s 32us/step - loss: 0.3977 - acc: 0.8128 - val_loss: 0.3935 - val_acc: 0.8165
Epoch 3/100
133332/133332 [==============================] - 4s 33us/step - loss: 0.3836 - acc: 0.8212 - val_loss: 0.3892 - val_acc: 0.8183
Epoch 4/100
133332/133332 [==============================] - 4s 33us/step - loss: 0.3729 - acc: 0.8268 - val_loss: 0.3860 - val_acc: 0.8200
Epoch 5/100
133332/133332 [==============================] - 5s 37us/step - loss: 0.3632 - acc: 0.8325 - val_loss: 0.3806 - val_acc: 0.8232
Epoch 6/100
133332/133332 [==============================] - 4s 32us/step - loss: 0.3558 - acc: 0.8364 - val_loss: 0.3841 - val_acc: 0.8221
Epoch 7/100
133332/133332 [==============================] - 4s 32us/step - loss: 0.3487 - acc: 0.8413 - val_loss: 0.3813 - val_acc: 0.8232
Epoch 8/100
133332/133332 [===

Epoch 9/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3429 - acc: 0.8437 - val_loss: 0.4023 - val_acc: 0.8092
Epoch 10/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3347 - acc: 0.8487 - val_loss: 0.4041 - val_acc: 0.8112
Epoch 00010: early stopping
66666/66666 [==============================] - 5s 71us/step
Model:  deep_HB
[0.81232375352492947, 0.81134311343471066, 0.81119311193469568]
0.81% (+/- 0.00%)
Negative sentiment: 77.92%  Positive sentiment: 84.40%
Percentage of positive classifications (should be 50%ish): 53.2410091906
Time taken:  3.234620213508606 

tweets processed: 0  of total number of tweets: 200000
tweets processed: 50000  of total number of tweets: 200000
tweets processed: 100000  of total number of tweets: 200000
tweets processed: 150000  of total number of tweets: 200000
Train on 133332 samples, validate on 66668 samples
Epoch 1/100
133332/133332 [==============================] - 18s 132us/step - loss: 0.4

Epoch 2/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3935 - acc: 0.8160 - val_loss: 0.3947 - val_acc: 0.8150
Epoch 3/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3790 - acc: 0.8239 - val_loss: 0.3911 - val_acc: 0.8158
Epoch 4/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3687 - acc: 0.8291 - val_loss: 0.3848 - val_acc: 0.8189
Epoch 5/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3593 - acc: 0.8339 - val_loss: 0.3812 - val_acc: 0.8203
Epoch 6/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3515 - acc: 0.8389 - val_loss: 0.3809 - val_acc: 0.8225
Epoch 7/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3445 - acc: 0.8429 - val_loss: 0.3809 - val_acc: 0.8229
Epoch 8/100
133334/133334 [==============================] - 5s 34us/step - loss: 0.3351 - acc: 0.8478 - val_loss: 0.3846 - val_acc: 0.8224
Epoch 9/100
133334/1

Epoch 5/100
133334/133334 [==============================] - 5s 37us/step - loss: 0.3767 - acc: 0.8251 - val_loss: 0.3964 - val_acc: 0.8140
Epoch 6/100
133334/133334 [==============================] - 5s 34us/step - loss: 0.3676 - acc: 0.8300 - val_loss: 0.3970 - val_acc: 0.8138
Epoch 7/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3610 - acc: 0.8334 - val_loss: 0.3986 - val_acc: 0.8126
Epoch 8/100
133334/133334 [==============================] - 4s 33us/step - loss: 0.3532 - acc: 0.8378 - val_loss: 0.3988 - val_acc: 0.8122
Epoch 00008: early stopping
66666/66666 [==============================] - 6s 91us/step
Train on 133334 samples, validate on 66666 samples
Epoch 1/100
133334/133334 [==============================] - 11s 80us/step - loss: 0.5024 - acc: 0.7571 - val_loss: 0.4238 - val_acc: 0.7987
Epoch 2/100
133334/133334 [==============================] - 5s 34us/step - loss: 0.4105 - acc: 0.8062 - val_loss: 0.4094 - val_acc: 0.8059
Epoch 3/100
133334/1

Epoch 9/100
133332/133332 [==============================] - 4s 34us/step - loss: 0.3362 - acc: 0.8481 - val_loss: 0.3830 - val_acc: 0.8228
Epoch 10/100
133332/133332 [==============================] - 4s 33us/step - loss: 0.3287 - acc: 0.8532 - val_loss: 0.3842 - val_acc: 0.8224
Epoch 00010: early stopping
66668/66668 [==============================] - 7s 102us/step
Train on 133334 samples, validate on 66666 samples
Epoch 1/100
133334/133334 [==============================] - 8s 64us/step - loss: 0.4918 - acc: 0.7610 - val_loss: 0.4049 - val_acc: 0.8081
Epoch 2/100
133334/133334 [==============================] - 5s 34us/step - loss: 0.3957 - acc: 0.8126 - val_loss: 0.3917 - val_acc: 0.8148
Epoch 3/100
133334/133334 [==============================] - 5s 34us/step - loss: 0.3832 - acc: 0.8198 - val_loss: 0.3868 - val_acc: 0.8170
Epoch 4/100
133334/133334 [==============================] - 4s 34us/step - loss: 0.3710 - acc: 0.8273 - val_loss: 0.3903 - val_acc: 0.8150
Epoch 5/100
133334/

In [13]:
print(len(accuracies))

18


In [14]:
# pickle the data
#simpleprocessing = { "corpus": corpuses, "accuracies":accuracies, "stds":stds, "names":names }
pickle.dump(simpleprocessing, open( "simpleprocessing.pkl", "wb" ))

In [18]:
# pickle the data
# FORMAT OF DATA: { "corpus": corpuses, "accuracies":accuracies, "stds":stds, "names":names }
jesus_processing = pickle.load(open( "simpleprocessing.pkl", "rb"))
print(jesus_processing['accuracies'])

[0.82133998906616545, 0.82472498886446033, 0.82494499361862905, 0.82164998541209167, 0.82181498871328384, 0.82209498941392889, 0.82197998751452073, 0.82209999246628351, 0.82158998616684054, 0.82122498791436582, 0.8116199929647786, 0.82247999341456168, 0.82239998601342779, 0.82525499011455372, 0.81319999466150128, 0.82106500371596403, 0.82086998511551501, 0.81494999296663673]


## Want to determine which preprocessing techniques that improved the accuracy, and keep them: 

In [15]:
corpuses_1=[]
names_1=[]
stds_1=[]
acc_1=[]
print('The original corpus gave accuracy of: ',accuracies[0],'\n')
for i in range(1,len(accuracies)):
    if accuracies[i]>=accuracies[0]:
        corpuses_1.append(corpuses[i])
        names_1.append(names[i])
        stds_1.append(stds[i])
        acc_1.append(accuracies[i])
        print('IMPROVED:  ',names[i],', score:',accuracies[i],'std:',stds[i])
    else:
        print('Not better:',names[i],', score:',accuracies[i],'std:',stds[i])
        

The original corpus gave accuracy of:  0.821339989066 

IMPROVED:   SH_corpus , score: 0.824724988864 std: 0.000803296554916
IMPROVED:   SHM_corpus , score: 0.824944993619 std: 0.000689043300155
IMPROVED:   H_corpus , score: 0.821649985412 std: 0.00109691743644
IMPROVED:   HK_corpus , score: 0.821814988713 std: 0.000952763449648
IMPROVED:   PS_corpus , score: 0.822094989414 std: 0.000778588361733
IMPROVED:   NS__corpus , score: 0.821979987515 std: 0.00109219124748
IMPROVED:   OS_corpus , score: 0.822099992466 std: 0.000534105963169
IMPROVED:   N_corpus , score: 0.821589986167 std: 0.000987553393163
Not better: NM_corpus , score: 0.821224987914 std: 0.000875414752358
Not better: ST_corpus , score: 0.811619992965 std: 0.000501387612544
IMPROVED:   SP_corpus , score: 0.822479993415 std: 0.000560695488719
IMPROVED:   E_corpus , score: 0.822399986013 std: 0.00102337342868
IMPROVED:   SN_corpus , score: 0.825254990115 std: 0.000746920476196
Not better: RS_corpus , score: 0.813199994662 std: 

In [16]:
print(names_1)

['SH_corpus', 'SHM_corpus', 'H_corpus', 'HK_corpus', 'PS_corpus', 'NS__corpus', 'OS_corpus', 'N_corpus', 'SP_corpus', 'E_corpus', 'SN_corpus']


## Round 2: apply best preprocessing technique to all other techniques that imporved the accuracy: 

## Want to determine which preprocessing technique-combinations that improved the accuracy, and keep them: 

## Round 3: apply best preprocessing technique-combination to all other techniques that imporved the accuracy: 

### Dynamic stopword list

In [ ]:
stopwords= CL.get_dynamic_stopwords(full_corpus, MinDf=0.01, MaxDf=0.99,sublinearTF=True,useIDF=False)

In [ ]:
stopword_corpus=CL.remove_stopwords(full_corpus, stopwords)

In [ ]:
corpuses.append(stopword_corpus)
names.append('stopword_corpus')

# Testing "best preprocessing" with full dataset: 

Som før for å lage en keggle! 

In [ ]:
final_corpus=TO.preprocess_corpus(full_corpus, segmentation_hash=True, hashtag=True, hashtag_mention=True, set_to_not=True,elongation=True)


In [ ]:
model_score=GV.classify_with_neural_networks(neural_nets, global_vectors, final_corpus, total_training_tweets, nr_pos_tweets, epochs=6, n_folds=3)

# Making kaggle submission: 

In [ ]:
kaggle_name="keggle_glove_13_12_full.csv"
#final_corpus=n_grams_corpus

pred= GV.get_prediction(NN.deep_HB, global_vectors, final_corpus, total_training_tweets, nr_pos_tweets,kaggle_name, epochs=6)

In [ ]:
print(sum(pred))

In [ ]:
import enchant
d = enchant.Dict("en_US")